## Deriving Laplace approximation

We assume that the considered learning process produces a model $p(y | x, \theta)$ that maps the example's feature vector $x$ to an output variable $y$, and the mapping comes from a family of models, parametrised by $\theta$, a $d$-dimensional vector, $\theta \in \mathcal{R}^d$.  The parameter vector might have a prior distribution, $P(\theta)$, which acts as a regulariser. The training procedure is performed on a dataset of $n$ examples, $\mathbb{D} = \{x, y\}_{i = 1}^n$ and finds the optimal parameter vector $\hat \theta$ as an optimum of the log-likelihood of the posterior, $L(\theta)$ (MAP estimate):
$$
\hat \theta = argmax_{\theta} L(\theta) = argmax_{\theta} log \left[ P(\mathbb{D} | \theta) \cdot P(\theta) \right] = argmax_{\theta} \left[ \sum_{i = 1}^n log p(y_i|x_i, \theta) + log P (\theta) \right]
$$

After finding the MAP estimate $\hat \theta$, we make a Taylor expansion of the posterior likelihood in the neighbourhood of the local maximum $\hat \theta$ up to the second term:
$$
L(\theta) \approx L(\hat \theta) + \left( \nabla_{\hat \theta} L \right) (\theta - \hat \theta) + \frac{1}{2}(\theta - \hat \theta)^T \mathbb{H} (\theta - \hat \theta)
$$
, where $\mathbb{H}$ is the Hessian of $L$ calculated at the point $\theta = \hat \theta$.

However, since $\hat \theta$ is the local optimum, the first order term $\left( \nabla_{\hat \theta} L\right) (\theta - \hat \theta)$ equates to the zero vector. Hence,  we can re-write the previous equation as:
$$
L(\theta) = L(\hat \theta) + \frac{1}{2}(\theta - \hat \theta)^T \mathbb{H} (\theta - \hat \theta)
$$

We can treat the posterior $p(\theta| \mathbb{D})$ as a Normal distribution around the mode $\hat \theta$. Indeed, the log of the density function of the Normal distribution is a quadratic form, just as the one we obtained for the posterior $log p(\theta| \mathbb{D})$. Noting that $-\mathbb{H}$ is a positive-defined matrix (as  $\hat \theta$ is a local maximum), we obtain:
$$
p(\theta | \mathbb{D}) \propto exp \left[-\frac{1}{2}(\theta - \hat \theta)^T (-\mathbb{H}) (\theta - \hat \theta) \right] \Leftrightarrow \theta \sim \mathbb{N}(\hat \theta; \left( - \mathbb{H}\right)^{-1}) 
$$

Operating with a full Hessian $\mathbb{H}$ is impractical in high-dimensional spaces, as the number of entries in it grows quadratically with the number of dimensions $d$. An accepted approach is to approximate it with its diagonal (note, that if $-\mathbb{H}$ is positive-defined matrix, then the matrix $-diag(\mathbb{H})$ is also positive-defined). Denoting $\mathbb{A} = -diag(\mathbb{H})$, we obtain the Laplace approximation of the posterior:
$$
\theta \sim \mathbb{N}(\hat \theta; \mathbb{A}^{-1}) 
$$


## Estimating the diagonal of the Hessian

Right, but how do we calculate the Hessian? For the relatively simple linear models, such as logistic regression or least squares models with L2 regularisation, we can calculate the Hessian of the loss at the MAP solution directly summing of the points in the dataset $\{(y_i, x_i)\}_{i=1}^N$. Indeed, in the case of logistic regression without regularisation, the $j$th component of the Hessian's diagonal would be:
$$
diag(\mathbb{H})_j = \sum_{n = 1}^N p(y_i | x_i) \left(1 - p(y_i | x_i)\right) (x_i^j)^2
$$
To account for the regularisation, one needs to add the regularisation coefficient to each element of the diagonal.


In other cases, such as multi-layer deep net, it can be less trivial. You can still do that either by auto-differentiating gradients returned by your favourite auto-differentiating software (autograd, tensorflow, etc), or in  some smarter ways (check the chapter on Bayesian neural networks Bishop's book).

However, we can estimate it by sampling points from the dataset. Firstly, let's find the expectation of the negative log-likelihood's Hessian. Practically, it differs from the Hessian discussed above by a factor of $N$ or is approximated by the Hessian of the negative log-likelihood normalised by the number of examples.

$$
\mathbb{E}_{(x,y)} \left(-\frac{\partial^2 log p(x, y)}{\partial \theta^2} \right) = 
\mathbb{E}_{(x,y)} \left(-\frac{\partial \left[ \frac{1}{p(x, y)} \frac{\partial p(x,y)}{\partial \theta} \right]}{\partial \theta^2} \right) =
\mathbb{E}_{(x,y)} \left[ -\frac{1}{p(x, y)} \frac{\partial^2 p(x,y)}{\partial \theta^2} + 
\left( \frac{1}{p(x, y)} \frac{\partial p(x,y)}{\partial \theta} \right)\left( \frac{1}{p(x, y)} \frac{\partial p(x,y)}{\partial \theta} \right)^T \right] =
$$
$$
= -\sum_{x,y}p(x,y) \frac{1}{p(x, y)}\frac{\partial^2 p(x,y)}{\partial \theta^2} + \mathbb{E}_{(x,y)} \left[
\left( \frac{\partial log p(x,y)}{\partial \theta} \right)\left( \frac{\partial log p(x,y)}{\partial \theta} \right)^T \right]
$$
By exchanging the differentiation and summing, we obtain:
$$
\frac{\partial^2 \sum_{x,y} p(x,y)}{\partial \theta^2} + \mathbb{E}_{(x,y)} \left[
\left( \frac{\partial log p(x,y)}{\partial \theta} \right) \left( \frac{\partial log p(x,y)}{\partial \theta} \right)^T \right] = \mathbb{E}_{(x,y)} \left[
\left( \frac{\partial log p(x,y)}{\partial \theta} \right)\left( \frac{\partial log p(x,y)}{\partial \theta} \right)^T \right]
$$
Remember, $\theta$ only parameterises the conditional distribution of $y$ given $x$, $p(y | x, \theta)$. Re-writing $log p(x,y)$ as $log p(x,y) = log p(y | x) + log p(y) = log p(y | x, \theta) + log p(x)$, we obtain:

$$
\mathbb{E}_{(x,y)} \left[
\left( \frac{\partial log p(x,y)}{\partial \theta} \right)^T\left( \frac{\partial log p(x,y)}{\partial \theta} \right) \right] = \mathbb{E}_{(x,y)} \left[
\left( \frac{\partial log p(y| x, \theta)}{\partial \theta} \right)^T\left( \frac{\partial log p(y | x, \theta)}{\partial \theta} \right) \right]
$$

We can Monte-Carlo estimate this expression by sampling datapoints $S$:

$$
\mathbb{E}_{(x,y)} \left[
\left( \frac{\partial log p(y| x, \theta)}{\partial \theta} \right)\left( \frac{\partial log p(y | x, \theta)}{\partial \theta} \right)^T \right] \approx \frac{1}{|S|} \sum_{x \in S, y \sim p(y | x, \theta)}
\left( \frac{\partial log p(y| x, \theta)}{\partial \theta} \right) \left( \frac{\partial log p(y| x, \theta)}{\partial \theta} \right)^T
$$

We also notice that in the case of the logistic regression, the last expression actually gives us the same solution as the analytic one:
$$
\mathbb{E}_{(x,y)} \left[
\left( \frac{\partial log p(y| x, \theta)}{\partial \theta} \right)\left( \frac{\partial log p(y | x, \theta)}{\partial \theta} \right)^T \right] = \mathbb{E}_{x} \mathbb{E}_{y|x} (1 - p(y|x, \theta))^2 x x^T =
$$
$$
\mathbb{E}_{x} \left( p(y = 0|x, \theta) p(y = 1|x, \theta)^2 + p(y = 1|x, \theta) p(y = 0|x, \theta)^2 \right) x x^T  = \mathbb{E}_{x} \left( p(y = 0|x, \theta) p(y = 1|x, \theta)\right) x x^T = 
$$
$$
\mathbb{E}_{x} \left[ p(y|x, \theta) (1 - p(y|x, \theta)) x x^T \right] \approx \frac{1}{N} \sum_{i = 1}^N  \left[ p(y|x_i, \theta) (1 - p(y|x_i, \theta)) x_i x_i^T \right]
$$

% TODO: why expectation wrt y, not just sampling

% TODO: column-vectors